## LED点灯を題材にどのようにしてPS(Linux)側からPL(FPGA)側へと指示を出しているかを調べたノートブックです  
In[2], In[3], In[5]は全て同じ、LED0とLED１は光らせ、LED2とLED3は消すプログラムです。  書き方を変えているだけです。

In [1]:
#デフォルト回路の読み込み
from pynq import Overlay
# load Base Overlay
Overlay("/home/xilinx/pynq/bitstream/base.bit").download()

### LEDクラスを使うことで簡単にLEDを操作することが出来る。

In [2]:
from pynq.board import LED#PYNQ上の"xilinx/pynq/board/led.py"にある。
led0 = LED(0)
led1 = LED(1)
led2 = LED(2)
led3 = LED(3)

led0.on()
led1.on()
led2.off()
led3.off()

### led.py中身をとりだし、LEDクラスを用いずにLEDを点灯させてみる。
コードは以下のようになる。

In [3]:
from pynq import MMIO#メモリ操作のためのクラス(PYNQ上の"xilinx/pynq/mmio.py”にある。)
from pynq import PL#回路情報の取得のためのクラス(PYNQ上の"xilinx/pynq/pl.py"にある。)

LEDS_OFFSET0 = 0x8
LEDS_OFFSET1 = 0xC#12
led = MMIO(int(PL.ip_dict["SEG_swsleds_gpio_Reg"][0],16),16)
#　PL.ip_dict["SEG_swsleds_gpio_Reg"]で回路の情報を取得する事ができる。今回は"SEG_swsleds_gpio_Reg"に対応するベースアドレスを取得している。
#　MMIOはデバイスファイル/dev/memを書き換えるための関数である。

led.write(LEDS_OFFSET1, 0x0)
led.write(LEDS_OFFSET0, 0B0011) #0B(led3)(led2)(led1)(led0) メモリの特定のアドレスにLEDの状態を書き込む

PL.ip_dictについて詳しく見てみる。

In [4]:
#ip_dictは辞書。使用するピンの名前(SEG_swsleds_gpio_Reg)と
#対応するメモリの情報(['0x41200000', '0x00010000', None])がセットになっている
from pynq import PL
PL.ip_dict["SEG_swsleds_gpio_Reg"]

['0x41200000', '0x00010000', None]

PYNQではbitファイルの読み込みとともに、tclファイルの情報を解析し、ip_dictという名前の辞書が作成される。回路名とそのアドレス、オフセット、状態(デフォルトはNone)が一つにまとまっており、回路の名前からアドレスやその他の情報を調べることが出来る。  
公式ドキュメント  
http://pynq.readthedocs.io/en/latest/16_creating_overlays.html  

以下は"xilinx/pynq/bitstream/base.tcl"の一部を切り取り、並び替えたもの
```tcl
3136   create_bd_addr_seg -range 0x00010000 -offset 0x41200000 [get_bd_addr_spaces processing     _system7_0/Data] [get_bd_addr_segs swsleds_g     pio/S_AXI/Reg] SEG_swsleds_gpio_Reg
...
3129   create_bd_addr_seg -range 0x00010000 -offs et 0x41210000 [get_bd_addr_spaces processing     _system7_0/Data] [get_bd_addr_segs btns_gpio     /S_AXI/Reg] SEG_btns_gpio_Reg
...
3127   create_bd_addr_seg -range 0x00010000 -offset 0x41220000 [get_bd_addr_spaces processing     _system7_0/Data] [get_bd_addr_segs video/axi     _gpio_video/S_AXI/Reg] SEG_axi_gpio_video_Reg
...
```
tclファイルはbitファイルと合わせて生成される。PL回路の情報を含むファイル。

### さらにmmio.pyの中身を取り出し、MMIOクラスを使わずにLEDの操作をするコードをつくる。

In [5]:
#　led.write(LEDS_OFFSET0, 0B0011) の中身
#以下general_const.pyにて定義されている変数群
# MMIO constants
MMIO_FILE_NAME = '/dev/mem'#デバイスファイルの場所
MMIO_WORD_LENGTH = 4
MMIO_WORD_MASK = ~(MMIO_WORD_LENGTH - 1) 
#mmio.pyの中ではfrom . import general_constatとして上記の変数を読み込んでいる

import os
import mmap
import struct
length = 16
base_addr = int('0x41200000',16) #このアドレスは既知のものとする。
virt_base = base_addr & ~(mmap.PAGESIZE - 1)
virt_offset = base_addr - virt_base

f = os.open(MMIO_FILE_NAME, os.O_RDWR | os.O_SYNC)
mem = mmap.mmap(f, (length + virt_offset), 
                            mmap.MAP_SHARED, 
                            mmap.PROT_READ | mmap.PROT_WRITE,
                            offset = virt_base)
offset = 0x8#LEDS_OFFSET0
data = 0B0011#0B(led3)(led2)(led1)(led0)LEDの状態
mem.seek((virt_offset + offset) & MMIO_WORD_MASK)#Set the file’s current position. 
mem.write(struct.pack('I', data)) # I:unsigned int


mmapに関するメモ
>Linuxでは、/dev/memというスペシャルファイルを経由して、メモリ空間の任意の物理アドレスにアクセスできるからです。...Linuxのメモリ空間は仮想メモリ空間なので、...mmapという関数を使ってマップしてやる必要があります。
http://d.hatena.ne.jp/ymko/20130904/p1
http://www.mech.tohoku-gakuin.ac.jp/rde/contents/linux/drivers/nodriver.html

## まとめ
メモリを介してPSとPLがつながっていることがわかった。  また、bitファイルが回路生成用いられ、tclファイルはPythonがbitファイルの回路を解析するために用いられることがわかった。  
これまでの一連の流れを図にしてみた。
![](./LED.png)
##### 参考にした方のブログから引用
>FPGAの制御（FPGAのレジスタへの書き込み）はmmio.pyのMMIOクラスを使って、/dev/memデバイスファイル経由でメモリー空間にマップされたFPGAのレジスタにアクセスしているようです。  
http://todotani.cocolog-nifty.com/blog/2017/01/pynq-z1overlayp.html

#### PYNQコミッターもこのように書いています。
>What is an overlay? The overlay is a design that’s loaded into the Zynq SoC’s programmable logic (PL). The overlay can be designed to accelerate a function in the programmable logic or provide an interfacing capability using the PL. In short, overlays give Pynq its, unique capabilities.
 
>What is important to understand about the overlay is that** there is not a Python-to-PL high-level synthesis process involved.** Instead, we develop the overlay using one of the standard Xilinx design methodologies (SDSoC, Vivado, or Vivado HLS). Once we’ve created the bit file for the overlay, we then integrate it within the Pynq architecture and establish the required parameters to communicate with it using Python.

>Like all things with the Zynq SoC that we have looked at to date, this is very simple. We can easily integrate with the Python environment using the bit file and other files provided with the Vivado build.** We do this with the Python MMIO class, which allows us to interact with designs in the PL through memory-mapped reads and writes. ** The memory map of the current overlay in the PL is all we need. Of course, we can change the contents of the PL on the fly as our application requires to accelerate functions in the PL.
https://forums.xilinx.com/t5/Xcell-Daily-Blog/Adam-Taylor-s-MicroZed-Chronicles-Part-156-Pynq-Hardware/ba-p/732835

### 参考  
http://todotani.cocolog-nifty.com/blog/2017/01/pynq-z1overlayp.html  
https://forums.xilinx.com/t5/Xcell-Daily-Blog/Adam-Taylor-s-MicroZed-Chronicles-Part-156-Pynq-Hardware/ba-p/732835  
http://pynq.readthedocs.io/en/latest/16_creating_overlays.html  